In [5]:
import pandas as pd
from glob import glob
import numpy as np
import xml.etree.ElementTree as ET
import os
from typing import List,Union,Final

#### Q1 How much flow each retail brokers have

In [15]:
COLS : Final[str] = ["DataDate","category","venue",
 'orderPct',
 'marketPct',
 'marketableLimitPct',
 'nonMarketableLimitPct',
 'otherPct',
 'netPmtPaidRecvMarketOrdersUsd',
 'netPmtPaidRecvMarketOrdersCph',
 'netPmtPaidRecvMarketableLimitOrdersUsd',
 'netPmtPaidRecvMarketableLimitOrdersCph',
 'netPmtPaidRecvNonMarketableLimitOrdersUsd',
 'netPmtPaidRecvNonMarketableLimitOrdersCph',
 'netPmtPaidRecvOtherOrdersUsd',
 'netPmtPaidRecvOtherOrdersCph']
def readXMLToList(root,datarows:List[List[Union[int,float,str]]]) -> None:
    for monthly in root.findall("rMonthly"):
        for venue in monthly.find("rSP500/rVenues"):
            this_row= []
            this_row.append(int(monthly.find("year").text+monthly.find('mon').text))
            this_row.append("SP500")
            for field in venue:
                if field.tag in {"materialAspects","mic"}:
                    pass
                elif field.tag=="name":
                    this_row.append(field.text)
                else:
                    this_row.append(float(field.text))
            datarows.append(this_row)
        for venue in monthly.find("rOtherStocks/rVenues"):
            this_row= []
            this_row.append(int(monthly.find("year").text+monthly.find('mon').text))
            this_row.append("OtherStocks")
            for field in venue:
                if field.tag in {"materialAspects","mic"}:
                    pass
                elif field.tag=="name":
                    this_row.append(field.text)
                else:
                    this_row.append(float(field.text))
            datarows.append(this_row)
        for venue in monthly.find("rOptions/rVenues"):
            this_row= []
            this_row.append(int(monthly.find("year").text+monthly.find('mon').text))
            this_row.append("Options")
            for field in venue:
                if field.tag in {"materialAspects","mic"}:
                    pass
                elif field.tag=="name":
                    this_row.append(field.text)
                else:
                    this_row.append(float(field.text))
            datarows.append(this_row)
def readByBroker(broker:str,report_path:str,):
    broker_report_path = os.path.join(report_path,broker)
    broker_606_paths = glob(os.path.join(broker_report_path,"*"))
    datarows = []
    for path in broker_606_paths:
        tree = ET.parse(path)
        root = tree.getroot()
        readXMLToList(root,datarows)
    res = pd.DataFrame(data=datarows,columns=COLS)
    res.insert(0,"broker",broker)
    return res

In [16]:
retail_brokers = ["Ameritrade","Chas","Etrade","Fidelity"]
report_path = os.path.abspath(os.path.join(".","downloaded_reports"))

In [20]:
reports606_list = []
for broker in retail_brokers:
    reports606_list.append(readByBroker(broker,report_path))

In [21]:
report606 = pd.concat(reports606_list,axis=0)

In [22]:
report606

,broker,DataDate,category,venue,orderPct,marketPct,marketableLimitPct,nonMarketableLimitPct,otherPct,netPmtPaidRecvMarketOrdersUsd,netPmtPaidRecvMarketOrdersCph,netPmtPaidRecvMarketableLimitOrdersUsd,netPmtPaidRecvMarketableLimitOrdersCph,netPmtPaidRecvNonMarketableLimitOrdersUsd,netPmtPaidRecvNonMarketableLimitOrdersCph,netPmtPaidRecvOtherOrdersUsd,netPmtPaidRecvOtherOrdersCph
0,Ameritrade,202101,SP500,"Citadel Securities, LLC",41.43,47.86,46.29,25.36,52.91,773557.0,12.0000,325186.0,12.0000,613192.0,33.8500,157277.0,9.4700
1,Ameritrade,202101,SP500,"Virtu Americas, LLC",20.65,24.57,22.10,4.67,34.54,384486.0,12.0000,154965.0,12.0000,174429.0,33.9600,92009.0,9.3100
2,Ameritrade,202101,SP500,"UBS Securities, LLC",15.11,1.31,5.12,39.44,1.03,20619.0,12.0000,15716.0,12.0000,550729.0,32.4300,1276.0,12.4900
3,Ameritrade,202101,SP500,"G1 Execution Services, LLC",12.74,24.81,23.68,8.64,6.45,408386.0,12.0000,156937.0,12.0000,326188.0,36.6000,24925.0,12.4600
4,Ameritrade,202101,SP500,"Two Sigma Securities, LLC",7.80,0.96,2.32,20.13,0.66,14885.0,12.0000,9578.0,12.0000,439692.0,33.5700,877.0,12.3600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,Fidelity,202306,Options,Citadel Securities LLC,46.51,54.85,54.18,41.31,52.19,579762.0,9.0682,516395.0,8.5035,2142932.0,36.7342,-15044.0,-0.1989
1052,Fidelity,202306,Options,Wolverine Execution Services LLC,18.54,1.90,3.01,30.63,0.00,25011.0,10.7977,39420.0,12.7579,991736.0,24.7221,0.0,0.0000
1053,Fidelity,202306,Options,Dash Financial Technologies LLC,15.97,5.63,6.17,20.82,16.01,84536.0,11.9881,87550.0,12.7013,851818.0,31.4427,339822.0,11.5795
1054,Fidelity,202306,Options,Susquehanna Capital Group,14.25,32.11,30.54,2.65,28.55,289991.0,7.3685,406258.0,11.3766,114721.0,24.6309,754211.0,17.1445
